In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split


In [2]:
def load_videos_from_folder(folder, label, num_frames=20, img_size=(64,64)):
    data = []
    labels = []
    for file in os.listdir(folder):
        if file.endswith(".mp4"):
            path = os.path.join(folder, file)
            cap = cv2.VideoCapture(path)

            frames = []
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_idxs = np.linspace(0, total_frames-1, num_frames).astype(int)

            for idx in frame_idxs:
                cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                ret, frame = cap.read()
                if ret:
                    frame = cv2.resize(frame, img_size)
                    frame = frame / 255.0  # normalize
                    frames.append(frame)

            cap.release()

            if len(frames) == num_frames:
                data.append(frames)
                labels.append(label)

    return np.array(data), np.array(labels)

# Load autism videos (label 1) and non-autism (label 0)
autism_data, autism_labels = load_videos_from_folder("newdataset/autismdataset", 1)
nonautism_data, nonautism_labels = load_videos_from_folder("newdataset/nonautismdataset", 0)

X = np.concatenate([autism_data, nonautism_data], axis=0)
y = np.concatenate([autism_labels, nonautism_labels], axis=0)

print("Dataset shape:", X.shape, y.shape)  # (samples, num_frames, 64,64,3)


Dataset shape: (69, 20, 64, 64, 3) (69,)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [4]:
def build_model(input_shape=(20,64,64,3)):
    model = models.Sequential([
        layers.TimeDistributed(layers.Conv2D(32, (3,3), activation='relu'), input_shape=input_shape),
        layers.TimeDistributed(layers.MaxPooling2D((2,2))),
        layers.TimeDistributed(layers.Conv2D(64, (3,3), activation='relu')),
        layers.TimeDistributed(layers.MaxPooling2D((2,2))),
        layers.TimeDistributed(layers.Flatten()),
        layers.LSTM(64),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
model.summary()


e:\MINI PROJECT\final.final demo\autism_env\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 20, 62, 62, 32) │           896 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 20, 31, 31, 32) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 20, 29, 29, 64) │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 20, 14, 14, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 20, 12544)      │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │     3,227,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,251,521 (12.40 MB)

 Trainable params: 3,251,521 (12.40 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=4
)


Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 449ms/step - accuracy: 0.7455 - loss: 0.6436 - val_accuracy: 0.7857 - val_loss: 0.5388
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 377ms/step - accuracy: 0.7455 - loss: 0.5596 - val_accuracy: 0.7857 - val_loss: 0.5669
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 377ms/step - accuracy: 0.7455 - loss: 0.5727 - val_accuracy: 0.7857 - val_loss: 0.5483
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 379ms/step - accuracy: 0.7455 - loss: 0.5630 - val_accuracy: 0.7857 - val_loss: 0.5435
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 379ms/step - accuracy: 0.7455 - loss: 0.5576 - val_accuracy: 0.7857 - val_loss: 0.5672
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 377ms/step - accuracy: 0.7455 - loss: 0.5355 - val_accuracy: 0.7857 - val_loss: 0.5849
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 377ms/step - accuracy: 0.7273 - loss: 0.5693 - val_accuracy: 0.6429 - val_loss: 0.6442
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 393ms/step - accuracy: 0.7455 - loss: 0.5264 - val_accuracy: 0

In [6]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

# Predict single video
pred = model.predict(X_test[:1])
print("Prediction:", "Autism" if pred[0] > 0.5 else "Non-Autism")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.6429 - loss: 0.6821
Test Accuracy: 64.29%
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 805ms/step
Prediction: Autism


In [8]:
# After training
model.save("video_autism_model.h5")   # <-- save with your chosen name


In [5]:
import cv2
import numpy as np
import tensorflow as tf

# -----------------------------
# 1. Load trained model
# -----------------------------
model = tf.keras.models.load_model("video_autism_model.h5")

# -----------------------------
# 2. Preprocess video for prediction
# -----------------------------
def preprocess_video(video_path, num_frames=20, img_size=(64,64)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_idxs = np.linspace(0, total_frames-1, num_frames).astype(int)

    for idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, img_size)
            frame = frame / 255.0
            frames.append(frame)

    cap.release()
    if len(frames) == num_frames:
        return np.array([frames])
    else:
        return None

# -----------------------------
# 3. Prediction + Show Video
# -----------------------------
def predict_and_show(video_path):
    data = preprocess_video(video_path)
    if data is None:
        print("❌ Not enough frames extracted!")
        return
    
    pred = model.predict(data)
    label = "Autism" if pred[0][0] > 0.5 else "Non-Autism"
    confidence = pred[0][0] if pred[0][0] > 0.5 else 1 - pred[0][0]

    print(f"Prediction: {label} (Confidence: {confidence:.2f})")

    # OpenCV video display
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Overlay prediction text
        text = f"{label} ({confidence:.2f})"
        cv2.putText(frame, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow("Prediction Output", frame)

        # Quit with 'q'
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# -----------------------------
# 4. Run on your video
# -----------------------------
video_path = r"E:\MINI PROJECT\final.final demo\newdataset\autismdataset\6 month old headbanging sad but true.mp4"
predict_and_show(video_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Autism (Confidence: 0.77)
